# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 10 2022 8:58AM,250465,16,8754672,Sartorius Bioprocess Solutions,Released
1,Nov 10 2022 8:58AM,250465,16,8754671,Sartorius Bioprocess Solutions,Released
2,Nov 10 2022 8:58AM,250465,16,8754676,Sartorius Bioprocess Solutions,Released
3,Nov 10 2022 8:58AM,250465,16,8755214,Sartorius Bioprocess Solutions,Released
4,Nov 10 2022 8:58AM,250466,16,8754574,Sartorius Lab Products and Service,Released
5,Nov 10 2022 8:54AM,250462,10,Enova-10584-BO,Emerginnova,Released
6,Nov 10 2022 8:54AM,250462,10,Eme-104624-BO,Emerginnova,Released
7,Nov 10 2022 8:54AM,250462,10,Enova-10789-BO,Emerginnova,Released
8,Nov 10 2022 8:54AM,250462,10,Enova-10832-BO,Emerginnova,Released
9,Nov 10 2022 8:54AM,250462,10,Enova-10841,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,250459,Released,1
21,250462,Released,22
22,250463,Released,1
23,250465,Released,4
24,250466,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
250459,NaN,1.0
250462,NaN,22.0
250463,NaN,1.0
250465,NaN,4.0
250466,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
250387,0.0,1.0
250388,0.0,1.0
250390,0.0,1.0
250391,0.0,1.0
250405,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
250387,0,1
250388,0,1
250390,0,1
250391,0,1
250405,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,250387,0,1
1,250388,0,1
2,250390,0,1
3,250391,0,1
4,250405,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,250387,,1
1,250388,,1
2,250390,,1
3,250391,,1
4,250405,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 10 2022 8:58AM,250465,16,Sartorius Bioprocess Solutions
4,Nov 10 2022 8:58AM,250466,16,Sartorius Lab Products and Service
5,Nov 10 2022 8:54AM,250462,10,Emerginnova
27,Nov 10 2022 8:39AM,250463,20,"ACI Healthcare USA, Inc."
28,Nov 10 2022 8:38AM,250450,10,ISDIN Corporation
38,Nov 10 2022 8:36AM,250459,16,"SHL Pharma, LLC"
39,Nov 10 2022 8:33AM,250456,12,Hush Hush
52,Nov 10 2022 4:17AM,250449,10,ISDIN Corporation
78,Nov 10 2022 4:14AM,250448,10,ISDIN Corporation
107,Nov 10 2022 4:12AM,250447,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Nov 10 2022 8:58AM,250465,16,Sartorius Bioprocess Solutions,,4
1,Nov 10 2022 8:58AM,250466,16,Sartorius Lab Products and Service,,1
2,Nov 10 2022 8:54AM,250462,10,Emerginnova,,22
3,Nov 10 2022 8:39AM,250463,20,"ACI Healthcare USA, Inc.",,1
4,Nov 10 2022 8:38AM,250450,10,ISDIN Corporation,,10
5,Nov 10 2022 8:36AM,250459,16,"SHL Pharma, LLC",,1
6,Nov 10 2022 8:33AM,250456,12,Hush Hush,11,2
7,Nov 10 2022 4:17AM,250449,10,ISDIN Corporation,,26
8,Nov 10 2022 4:14AM,250448,10,ISDIN Corporation,,29
9,Nov 10 2022 4:12AM,250447,10,ISDIN Corporation,,13


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 10 2022 8:58AM,250465,16,Sartorius Bioprocess Solutions,4,
1,Nov 10 2022 8:58AM,250466,16,Sartorius Lab Products and Service,1,
2,Nov 10 2022 8:54AM,250462,10,Emerginnova,22,
3,Nov 10 2022 8:39AM,250463,20,"ACI Healthcare USA, Inc.",1,
4,Nov 10 2022 8:38AM,250450,10,ISDIN Corporation,10,
5,Nov 10 2022 8:36AM,250459,16,"SHL Pharma, LLC",1,
6,Nov 10 2022 8:33AM,250456,12,Hush Hush,2,11
7,Nov 10 2022 4:17AM,250449,10,ISDIN Corporation,26,
8,Nov 10 2022 4:14AM,250448,10,ISDIN Corporation,29,
9,Nov 10 2022 4:12AM,250447,10,ISDIN Corporation,13,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 10 2022 8:58AM,250465,16,Sartorius Bioprocess Solutions,4,
1,Nov 10 2022 8:58AM,250466,16,Sartorius Lab Products and Service,1,
2,Nov 10 2022 8:54AM,250462,10,Emerginnova,22,
3,Nov 10 2022 8:39AM,250463,20,"ACI Healthcare USA, Inc.",1,
4,Nov 10 2022 8:38AM,250450,10,ISDIN Corporation,10,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Nov 10 2022 8:58AM,250465,16,Sartorius Bioprocess Solutions,4,NaN
1,Nov 10 2022 8:58AM,250466,16,Sartorius Lab Products and Service,1,NaN
2,Nov 10 2022 8:54AM,250462,10,Emerginnova,22,NaN
3,Nov 10 2022 8:39AM,250463,20,"ACI Healthcare USA, Inc.",1,NaN
4,Nov 10 2022 8:38AM,250450,10,ISDIN Corporation,10,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 10 2022 8:58AM,250465,16,Sartorius Bioprocess Solutions,4,0.0
1,Nov 10 2022 8:58AM,250466,16,Sartorius Lab Products and Service,1,0.0
2,Nov 10 2022 8:54AM,250462,10,Emerginnova,22,0.0
3,Nov 10 2022 8:39AM,250463,20,"ACI Healthcare USA, Inc.",1,0.0
4,Nov 10 2022 8:38AM,250450,10,ISDIN Corporation,10,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2754829,115,3.0
12,250456,2,11.0
15,250416,83,0.0
16,751390,6,0.0
20,500887,2,0.0
21,1001556,4,0.0
22,250443,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2754829,115,3.0
1,12,250456,2,11.0
2,15,250416,83,0.0
3,16,751390,6,0.0
4,20,500887,2,0.0
5,21,1001556,4,0.0
6,22,250443,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,115,3.0
1,12,2,11.0
2,15,83,0.0
3,16,6,0.0
4,20,2,0.0
5,21,4,0.0
6,22,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,115.0
1,12,Released,2.0
2,15,Released,83.0
3,16,Released,6.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,20,21,22
Status,,,,,,,
Executing,3.0,11.0,0.0,0.0,0.0,0.0,0.0
Released,115.0,2.0,83.0,6.0,2.0,4.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,20,21,22
0,Executing,3.0,11.0,0.0,0.0,0.0,0.0,0.0
1,Released,115.0,2.0,83.0,6.0,2.0,4.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,20,21,22
0,Executing,3.0,11.0,0.0,0.0,0.0,0.0,0.0
1,Released,115.0,2.0,83.0,6.0,2.0,4.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()